<a href="https://colab.research.google.com/github/dikoharyadhanto/GIS_for_Python_Documentation/blob/main/Geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-profiling
!pip install pandas
!pip install googlemaps
!pip install gmaps
!pip install KeplerGl
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 21.6 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=bdc1e306964b7828166b10b0fc1daf0bd4f7b886f755fae00600d11f2acecd14
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40713 sha256=91e9eadd11247da1c5b2d7e6bf1e641f860b787c55adba8bfe4d1

In [ ]:
# Load the required libraries
import pandas as pd
from pandas_profiling import ProfileReport
from googlemaps import Client as GoogleMaps
import googlemaps
import gmaps
from keplergl import KeplerGl
import geopandas as gpd

<ipython-input-4-4ed2b3e32c60>:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [33]:
# Load and explore the data
raw = pd.read_excel("/content/DATA 300 SURVEY TERMIN-1 update.xlsx", sheet_name='Raw Data')
addresses = raw[['Response ID', '106. Alamat Responden']]
# Look at the top few rows
addresses.head()

,Response ID,106. Alamat Responden
0,80475576,Cowang Dereng Labuan Bajo
1,80712619,"Lancang, Kel. Wae Kelambuk RT. 20, RW.04"
2,80713701,Kompleks Telkom Labuan Bajo
3,80715760,Kaper
4,80716226,Labuan Bajo


In [34]:
addresses.describe()

,Response ID
count,3.030000e+02
mean,8.084261e+07
std,5.691891e+04
min,8.047558e+07
25%,8.082797e+07
50%,8.085736e+07
75%,8.088511e+07
max,8.090705e+07


In [35]:
addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Response ID            303 non-null    int64 
 1   106. Alamat Responden  303 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.9+ KB


In [36]:
# This is where we will need the API key
gmaps = googlemaps.Client(key='')

In [37]:
# Lets just pass the full address column to the google API
addresses1= addresses.iloc[:,-1:]
addresses1.head()

,106. Alamat Responden
0,Cowang Dereng Labuan Bajo
1,"Lancang, Kel. Wae Kelambuk RT. 20, RW.04"
2,Kompleks Telkom Labuan Bajo
3,Kaper
4,Labuan Bajo


In [38]:
addresses1['long'] = ""
addresses1['lat'] = ""

In [43]:
addresses1['Region'] = 'Labuan Bajo'
addresses1['Country'] = 'Indonesia'


In [46]:
addresses1['Full_Address'] = addresses1['106. Alamat Responden'].astype(str) + ', ' + \
                addresses1['Region'] + ', ' + \
                addresses1['Country']

In [47]:
addresses1.head()

,106. Alamat Responden,long,lat,Region,Country,Full_Address
0,Cowang Dereng Labuan Bajo,,,Labuan Bajo,Indonesia,"Cowang Dereng Labuan Bajo, Labuan Bajo, Indonesia"
1,"Lancang, Kel. Wae Kelambuk RT. 20, RW.04",,,Labuan Bajo,Indonesia,"Lancang, Kel. Wae Kelambuk RT. 20, RW.04, Labu..."
2,Kompleks Telkom Labuan Bajo,,,Labuan Bajo,Indonesia,"Kompleks Telkom Labuan Bajo, Labuan Bajo, Indo..."
3,Kaper,,,Labuan Bajo,Indonesia,"Kaper, Labuan Bajo, Indonesia"
4,Labuan Bajo,,,Labuan Bajo,Indonesia,"Labuan Bajo, Labuan Bajo, Indonesia"


In [48]:
print(len(addresses1))

303


In [49]:
count_not_found = 0
for x in range(len(addresses1)):
    geocode_result = gmaps.geocode(addresses1['Full_Address'][x])
    if geocode_result:
        addresses1['lat'][x] = geocode_result[0]['geometry']['location']['lat']
        addresses1['long'][x] = geocode_result[0]['geometry']['location']['lng']
    else:
        print(f"No result for address: {addresses1['Full_Address'][x]}")
        count_not_found += 1

print(f"Total addresses not found: {count_not_found}")

Total addresses not found: 0


In [52]:
# Lets join the results with original file
addresses['Lat']=addresses1['lat']
addresses['Lon']= addresses1['long']
addresses['Full Address']= addresses1['Full_Address']
addresses.head()

<ipython-input-52-eadafc0e5727>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses['Lat']=addresses1['lat']
<ipython-input-52-eadafc0e5727>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses['Lon']= addresses1['long']
<ipython-input-52-eadafc0e5727>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,Response ID,106. Alamat Responden,Lat,Lon,Full Address
0,80475576,Cowang Dereng Labuan Bajo,-8.489651,119.893765,"Cowang Dereng Labuan Bajo, Labuan Bajo, Indonesia"
1,80712619,"Lancang, Kel. Wae Kelambuk RT. 20, RW.04",-8.453848,119.872842,"Lancang, Kel. Wae Kelambuk RT. 20, RW.04, Labu..."
2,80713701,Kompleks Telkom Labuan Bajo,-8.453848,119.872842,"Kompleks Telkom Labuan Bajo, Labuan Bajo, Indo..."
3,80715760,Kaper,-8.453848,119.872842,"Kaper, Labuan Bajo, Indonesia"
4,80716226,Labuan Bajo,-8.453848,119.872842,"Labuan Bajo, Labuan Bajo, Indonesia"


In [53]:
addresses.to_csv('data_bajo_XY.csv', index=False, encoding='utf-8')